In [2]:
pip install tenseal scikit-learn numpy

Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import tenseal as ts
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load dataset
data = load_breast_cancer()
X, y = data.data, data.target

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale data for better convergence
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predict on raw (unencrypted) test data
y_pred = model.predict(X_test)
accuracy_plain = accuracy_score(y_test, y_pred)

print(f"Accuracy on raw (unencrypted) data: {accuracy_plain:.4f}")


Accuracy on raw (unencrypted) data: 0.9737


In [6]:
import tenseal as ts

# Ensure the CKKS context is correctly initialized without the 'enable' argument
ckks_context = ts.context(
    scheme=ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[60, 40, 40, 60]
)

# Check if secret key is automatically generated or needs to be manually set
if not ckks_context.has_secret_key():
    ckks_context.generate_secret_key()

ckks_context.generate_galois_keys()
ckks_context.generate_relin_keys()

# Set scale after context is created
ckks_context.scale = 2**40  # Setting scale value


In [8]:
def encrypted_accuracy(X_test, y_test, model, encryption_scheme):
    correct = 0
    for i in range(len(X_test)):
        if encryption_scheme == "CKKS":
            # Ensure scale is correctly set
            enc_sample = ts.ckks_vector(ckks_context, X_test[i].tolist(), scale=2**40)
            enc_prediction = model.coef_.dot(enc_sample.decrypt(secret_key=ckks_context.secret_key())) + model.intercept_

        predicted_class = int((enc_prediction > 0).item())  # Ensures conversion to scalar
        
        if predicted_class == y_test[i]:
            correct += 1
            
    return correct / len(y_test)

# Compute accuracy
ckks_accuracy = encrypted_accuracy(X_test, y_test, model, "CKKS")

print(f"CKKS Encrypted Accuracy: {ckks_accuracy:.4f}")


CKKS Encrypted Accuracy: 0.9737
